In [ ]:
# !pip install transformers
# !pip install torch

import pandas as pd
from transformers import ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

# 데이터 로드
file_path = '/content/drive/MyDrive/SKT_FLY_AI/Project/lawsuit_type.csv'  # Colab에 업로드된 파일 경로
data = pd.read_csv(file_path, encoding='cp949')

In [ ]:
data

,Question,Answer,Label
0,중국에서 사채업을 하던 중국 국적의 갑이 대한민국에서 영업을 하기 위하여 대한민국에...,민사,0
1,갑 지방자치단체가 을 주식회사 소유의 토지가 포함된 부지에 대하여 제2종 지구단위계...,민사,0
2,지적을 제곱미터 단위까지만 표시하도록 한 지적법 같은법시행령 의 취지 및 1...,민사,0
3,전득자가 전매자를 대위하여 처분금지가처분결정을 받은 경우 그 효력범위,민사,0
4,근로자성 인정을 위한 사용종속관계 유무의 판단에 있어 그 고려 요소들의 입체적 파악,민사,0
...,...,...,...
25400,실체적 경합범 중 일부에 대하여 무죄 일부에 대하여 유죄를 선고한 항소심 판결에 대...,형사,1
25401,판결이유에서만 무죄의 판단이 있는 경우에도 형사보상을 청구할 수 있는지 여부,형사,1
25402,의장에 관련한 권리범위확인소송에서 당사자 사이의 합의의 효력이 미치는 범위,형사,1
25403,확정되지 아니한 판결의 집행에 대하여는 형사소송법 에 의한 이의신청을 할 수 있는가 여부,형사,1


In [ ]:
class LawsuitDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          padding='max_length',
          return_attention_mask=True,
          return_tensors='pt',
          truncation=True
        )

        return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(label, dtype=torch.long)
        }

MAX_LEN = 128  # 최대 시퀀스 길이
BATCH_SIZE = 16
tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")

ds = LawsuitDataset(
    texts=data['Question'].to_numpy(),
    labels=data['Label'].to_numpy(),
    tokenizer=tokenizer,
    max_len=MAX_LEN
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# !pip install transformers[torch]

In [ ]:
# pip install accelerate -U

In [ ]:
# !pip install accelerate -U

In [ ]:
# !pip show accelerate

In [ ]:
!pip install accelerate==0.20.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 3.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.1
    Uninstalling accelerate-0.20.1:
      Successfully uninstalled accelerate-0.20.1


In [ ]:
model = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=data['Label'].nunique())

training_args = TrainingArguments(
    output_dir='/content/model_results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    # eval_dataset=test_dataset  # 필요한 경우 검증 데이터셋을 제공하세요.
)

trainer.train()


pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
def predict(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    return probs.argmax().item()

new_text = "여기에 새로운 텍스트를 입력하세요."
predict(new_text)


In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 18.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.35.2', '0.26.1')

In [ ]:
from transformers import ElectraForPreTraining, ElectraTokenizer, ElectraForMaskedLM
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-generator")
generator = ElectraForMaskedLM.from_pretrained('monologg/koelectra-base-v3-generator')
discriminator = ElectraForPreTraining.from_pretrained("monologg/koelectra-base-v3-discriminator")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/463 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/149M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

In [ ]:
import torch

In [ ]:
# 텍스트를 준비합니다. 예를 들어, "[MASK]" 토큰을 사용할 수 있습니다.
text = "[CLS] 나는 오늘 [MASK] 갈 거야. [SEP]"
# 텍스트를 토큰화합니다.
input_ids = tokenizer.encode(text, return_tensors="pt")

# 마스크된 언어 모델을 사용하여 [MASK]의 토큰을 예측합니다.
outputs = generator(input_ids=input_ids, attention_mask=input_ids.ne(0))
predictions = outputs[0]

# 예측된 토큰의 ID를 실제 단어로 변환합니다.
masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
predicted_token_id = predictions[0, masked_index].argmax(axis=-1)
predicted_token = tokenizer.decode(predicted_token_id)

print(f"Original text: {text}")
print(f"Mask filled: {text.replace(tokenizer.mask_token, predicted_token)}")

Original text: [CLS] 나는 오늘 [MASK] 갈 거야. [SEP]
Mask filled: [CLS] 나는 오늘 안 갈 거야. [SEP]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments
import torch

# 1. 데이터 로드 및 전처리
# CSV 파일 로드
file_path = '/content/drive/MyDrive/SKT_FLY_AI/Project/lawsuit_type.csv'
df = pd.read_csv(file_path, encoding='cp949')

# "Question" 열을 텍스트 데이터로, "Label" 열을 레이블로 사용
texts = df["Question"].tolist()
labels = df["Label"].tolist()

# 2. 모델 및 토크나이저 로드
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=2)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# "Question" 열의 누락된 값이 있는지 확인
print("Missing values in 'Question':", df["Question"].isnull().sum())

# "Question" 열의 유형 확인
print("Type of 'Question':", type(df["Question"].iloc[0]))

# 처음 몇 개의 항목 출력
print("First few items in 'texts':", texts[:5])

Missing values in 'Question': 11
Type of 'Question': <class 'str'>
First few items in 'texts': ['중국에서 사채업을 하던 중국 국적의 갑이 대한민국에서 영업을 하기 위하여 대한민국에 입국하였는데 중국에서 부동산개발사업을 하던 중국 국적의 부부 을 등을 상대로 중국에서 이루어진 금전대여행위에 따른 대여금의 지급을 구하는 소를 대한민국 법원에 제기한 사안에서 대한민국 법원의 국제재판관할권이 인정된다고 한 사례', '갑 지방자치단체가 을 주식회사 소유의 토지가 포함된 부지에 대하여 제2종 지구단위계획구역 지정을 추진하였고 을 회사가 이를 신뢰하여 관련 절차를 진행하였으나 계획구역 지정이 철회되고 위 부지가 보전관리지역으로 지정되자 을 회사가 갑 지방자치단체를 상대로 신뢰보호의 원칙 위반을 이유로 손해배상을 구한 사안에서 갑 지방자치단체의 배상책임을 인정할 수 없다고 한 사례', '지적을 제곱미터 단위까지만 표시하도록 한 지적법    같은법시행령  의 취지 및 1필지 토지 중 일부에 대한 소유권이전등기절차이행판결 주문상에 지적이 제곱미터 미만의 단위까지 표시되어 있는 경우 판결경정의 대상이 되는지 여부', '전득자가 전매자를 대위하여 처분금지가처분결정을 받은 경우 그 효력범위', '근로자성 인정을 위한 사용종속관계 유무의 판단에 있어 그 고려 요소들의 입체적 파악']


In [ ]:
# "texts"의 각 항목이 문자열인지 확인하고, 그렇지 않은 항목을 처리
clean_texts = [str(text) for text in texts]

# 3. 데이터 토큰화
encoding = tokenizer(clean_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
input_ids = encoding["input_ids"]
attention_mask = encoding["attention_mask"]

# 4. 훈련 및 검증 데이터셋 분리
input_ids_train, input_ids_val, attention_mask_train, attention_mask_val, labels_train, labels_val = train_test_split(input_ids, attention_mask, labels, test_size=0.1)

# 5. 모델 훈련
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=100
)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: self.encodings[key][idx] for key in self.encodings}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset({'input_ids': input_ids_train, 'attention_mask': attention_mask_train}, labels_train)
val_dataset = Dataset({'input_ids': input_ids_val, 'attention_mask': attention_mask_val}, labels_val)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

# 6. 모델 평가 및 예측 (예시)
results = trainer.evaluate()
print(results)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
!pip install transformers
!pip install torch